<a href="https://colab.research.google.com/github/lily-larson/MGMT-467-Analytics-Portfolio/blob/main/Labs/Unit2_Lab1_GCS_BQ_Data_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
TQDM_DISABLE=1

# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [2]:
# Authenticate to Google Cloud in Colab
# This will prompt you to log in and select a project
from google.colab import auth
auth.authenticate_user()

# Set your Google Cloud Project ID and Region
# PROJECT_ID: Your unique GCP project identifier
# REGION: Google Cloud region for resources
import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["REGION"] = REGION
os.environ["JOB_VALIDATION_MODE"] = "DISABLED"
os.environ["TF_VALIDATION_MODE"] = "DISABLED"

# Set the gcloud config to your project
# This ensures subsequent gcloud commands use your project
!gcloud config set project $GOOGLE_CLOUD_PROJECT

# Print the set values for verification
print("Project:", PROJECT_ID, "| Region:", REGION)

# Done: Auth + Project/Region set

Enter your GCP Project ID: directed-bongo-471119-d1
Updated property [core/project].
Project: directed-bongo-471119-d1 | Region: us-central1


In [ ]:
# # EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
# # from google.colab import auth
# # auth.authenticate_user()
# #
# # import os
# # PROJECT_ID = input("Enter your GCP Project ID: ").strip()
# # REGION = "us-central1"  # keep consistent; change if instructed
# # os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
# # print("Project:", PROJECT_ID, "| Region:", REGION)
# #
# # # Set active project for gcloud/BigQuery CLI
# # !gcloud config set project $GOOGLE_CLOUD_PROJECT
# # !gcloud config get-value project
# # # Done: Auth + Project/Region set

### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [6]:
# Verify the active project and region
!gcloud config get-value project
import os
print("REGION:", os.environ.get("REGION"))

directed-bongo-471119-d1
REGION: us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

This ensures consisteny - all subsequent operations will be performed within the specified project and region. It also meets the dependencies of many Google Cloud commands and helps control costs that can vary by region. If you don't set your projectID and region at the top, commands may defualt to a different project or region and resources may be created in the wrong place. Besides being inconvenient and frustrating, this also impairs reproducibility.

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [ ]:
# Prompt the user to upload their Kaggle API token (kaggle.json)
# This file contains your Kaggle credentials and should be kept secure.
from google.colab import files
print("Upload your kaggle.json (Go to Kaggle -> Account -> Create New API Token)")
uploaded = files.upload()

# Save the uploaded file to the correct directory with secure permissions
# The .kaggle directory in the home folder is the default location for Kaggle config.
# Setting permissions to 0600 ensures only the file owner can read and write it,
# protecting your credentials from unauthorized access.
import os
os.makedirs('/root/.kaggle', exist_ok=True) # Create the directory if it doesn't exist
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # Set owner-only read/write permissions

# Verify the Kaggle CLI is installed and configured correctly by printing its version.
# This step helps ensure that subsequent Kaggle commands will work as expected,
# contributing to the reproducibility of the notebook.
!kaggle --version

Upload your kaggle.json (Go to Kaggle -> Account -> Create New API Token)


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


In [ ]:
# # EXAMPLE (from LLM) — Kaggle setup (commented)
# # from google.colab import files
# # print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
# # uploaded = files.upload()
# #
# # import os
# # os.makedirs('/root/.kaggle', exist_ok=True)
# # with open('/root/.kaggle/kaggle.json', 'wb') as f:
# #     f.write(uploaded[list(uploaded.keys())[0]])
# # os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only
# #
# # !kaggle --version

### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


In [ ]:
# Verify the Kaggle CLI is ready
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

0600 permissions allow only the file's owner to read and write to the file. If the file is available to other users, it is possible that they could use it to gain access to private information in my Kaggle account. These permissions protect me and my work.

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [ ]:
# Create the directory to store raw data
# -p flag ensures parent directories are created if they don't exist
# /content/data/raw is a standard location for raw data in Colab
!mkdir -p /content/data/raw

# Download the dataset from Kaggle
# -d specifies the dataset identifier (owner/dataset-name)
# -p specifies the path where the dataset zip file will be downloaded
# The Kaggle CLI uses the credentials configured in ~/.kaggle/kaggle.json
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data

# Unzip the downloaded dataset into the raw data directory
# -o flag allows overwriting existing files
# *.zip matches the downloaded zip file in /content/data
# -d specifies the destination directory for unzipping
!unzip -o /content/data/*.zip -d /content/data/raw

# List all CSV files in the raw data directory with their sizes
# -l uses a long listing format
# -h prints file sizes in human-readable format (e.g., KB, MB)
# *.csv matches all files with a .csv extension
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 1.42GB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M 

In [ ]:
# # EXAMPLE (from LLM) — Download & unzip (commented)
# # !mkdir -p /content/data/raw
# # !kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
# # !unzip -o /content/data/*.zip -d /content/data/raw
# # # List CSV inventory
# # !ls -lh /content/data/raw/*.csv

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


In [ ]:
# Verify the number of CSV files and print their names
import glob
csv_files = glob.glob('/content/data/raw/*.csv')
print("Found", len(csv_files), "CSV files:")
for csv_file in csv_files:
    print(csv_file)
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"

Found 6 CSV files:
/content/data/raw/recommendation_logs.csv
/content/data/raw/search_logs.csv
/content/data/raw/movies.csv
/content/data/raw/watch_history.csv
/content/data/raw/reviews.csv
/content/data/raw/users.csv


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

Clean file inventory allows easier data manipulation and reproducibility. It also makes troubleshooting, automating, and scripting easier.

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [ ]:
# Create a unique GCS bucket and upload the raw data
import uuid, os

# Generate a unique bucket name with a random suffix
# GCS bucket names must be globally unique. Using a random suffix helps avoid naming conflicts.
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name

# Create the GCS bucket
# --location=$REGION specifies the region for the bucket, which we set earlier.
# This command will fail if the bucket name is not globally unique, but the random suffix
# makes this unlikely.
!gcloud storage buckets create gs://$BUCKET_NAME --location=$REGION

# Upload all CSV files from the raw data directory to the bucket
# gs://$BUCKET_NAME/netflix/ creates a 'netflix' folder within the bucket
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/

# Print the bucket name for verification and future reference
print(f"Created and uploaded data to GCS bucket: {bucket_name}")

# Explain the benefits of staging data in GCS
print("\nBenefits of staging data in GCS:")
print("- Consistent and versionable source for BigQuery loads.")
print("- Decouples data source from BigQuery for flexible processing.")
print("- Leverages GCS's scalability, reliability, and integration with GCP services.")

Creating gs://mgmt467-netflix-8c398e66/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-8c398e66/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-8c398e66/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-8c398e66/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-8c398e66/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-8c398e66/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-8c398e66/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-8c398e66/netflix/watch_history.csv

Average throughput: 6.7MiB/s
Created and uploaded data to GCS bucket: mgmt467-netflix-8c398e66

Benefits of staging data in GCS:
- Consistent and versionable source for BigQuery loads.
- Decouples data source from BigQuery for flexible 

In [ ]:
# # EXAMPLE (from LLM) — GCS staging (commented)
# # import uuid, os
# # bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
# # os.environ["BUCKET_NAME"] = bucket_name
# # !gcloud storage buckets create gs://$BUCKET_NAME --location=$REGION
# # !gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
# # print("Bucket:", bucket_name)
# # # Verify contents
# # !gcloud storage ls gs://$BUCKET_NAME/netflix/

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [ ]:
# Verify the contents of the GCS bucket and show object sizes
# -l flag provides a long listing format, including size
# gs://$BUCKET_NAME/netflix/ specifies the path to list within the bucket
import os
!gcloud storage ls -l gs://$BUCKET_NAME/netflix/

      8002  2025-10-09T19:45:13Z  gs://mgmt467-netflix-8c398e66/netflix/README.md
    115942  2025-10-09T19:45:13Z  gs://mgmt467-netflix-8c398e66/netflix/movies.csv
   4695557  2025-10-09T19:45:15Z  gs://mgmt467-netflix-8c398e66/netflix/recommendation_logs.csv
   1861942  2025-10-09T19:45:15Z  gs://mgmt467-netflix-8c398e66/netflix/reviews.csv
   2250902  2025-10-09T19:45:15Z  gs://mgmt467-netflix-8c398e66/netflix/search_logs.csv
   1606820  2025-10-09T19:45:14Z  gs://mgmt467-netflix-8c398e66/netflix/users.csv
   9269425  2025-10-09T19:45:16Z  gs://mgmt467-netflix-8c398e66/netflix/watch_history.csv
TOTAL: 7 objects, 19808590 bytes (18.89MiB)


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

Staging the dataset in GCS creates a stable, traceable source for your data. This enables troubeshooting, reproducibility, and understandability. It also improves scalability and ensures your data persists even when the Colab session ends.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [ ]:
# # EXAMPLE (from LLM) — BigQuery dataset (commented)
# # DATASET="netflix"
# # # Attempt to create; ignore if exists
# # !bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

In [ ]:
# Create the BigQuery dataset if it doesn't exist
# bq mk -d creates a dataset
# --location=US specifies the multi-region location
# --description adds a description to the dataset
# || echo "Dataset may already exist." makes the command idempotent by
# suppressing the error if the dataset already exists and printing a friendly message instead.
DATASET = "netflix"
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

Dataset 'directed-bongo-471119-d1:netflix' successfully created.


In [ ]:
# # EXAMPLE (from LLM) — Load tables (commented)
# # tables = {
# #   "users": "users.csv",
# #   "movies": "movies.csv",
# #   "watch_history": "watch_history.csv",
# #   "recommendation_logs": "recommendation_logs.csv",
# #   "search_logs": "search_logs.csv",
# #   "reviews": "reviews.csv",
# # }
# # import os
# # for tbl, fname in tables.items():
# #   src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
# #   print("Loading", tbl, "from", src)
# #   !bq load --skip_leading_rows=1 --autodetect --source_format=CSV $DATASET.$tbl $src
# #
# # # Row counts
# # for tbl in tables.keys():
# #   !bq query --nouse_legacy_sql "SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `${GOOGLE_CLOUD_PROJECT}.netflix.{tbl}`".format(tbl=tbl)

In [ ]:
# Load tables from GCS into BigQuery
# --skip_leading_rows=1 skips the header row in the CSV files
# --autodetect allows BigQuery to automatically determine the schema
# --source_format=CSV specifies that the source files are in CSV format
# $DATASET.$tbl specifies the destination table in BigQuery
# $src specifies the source file in GCS
tables = {
  "users": "users.csv",
  "movies": "movies.csv",
  "watch_history": "watch_history.csv",
  "recommendation_logs": "recommendation_logs.csv",
  "search_logs": "search_logs.csv",
  "reviews": "reviews.csv",
}
import os
for tbl, fname in tables.items():
  src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
  print(f"Loading {tbl} from {src}")
  !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# Query row counts for each table
# --nouse_legacy_sql ensures standard SQL is used
# SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `${GOOGLE_CLOUD_PROJECT}.netflix.{tbl}` gets the table name and count
"""
print("\nRow counts:")
for tbl in tables.keys():
  query = f"SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.{tbl}`"
  !bq query --nouse_legacy_sql "{query}"
"""

Loading users from gs://mgmt467-netflix-8c398e66/netflix/users.csv
Waiting on bqjob_r3249c4cbfb72034f_00000199ca891af7_1 ... (1s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-8c398e66/netflix/movies.csv
Waiting on bqjob_r7c74b5b6ba693d97_00000199ca893848_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-8c398e66/netflix/watch_history.csv
Waiting on bqjob_rdb0eed03651de16_00000199ca8955fa_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-8c398e66/netflix/recommendation_logs.csv
Waiting on bqjob_r4a862037d7b479fa_00000199ca8977db_1 ... (1s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-8c398e66/netflix/search_logs.csv
Waiting on bqjob_r17ea650f7789fa80_00000199ca899473_1 ... (1s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-8c398e66/netflix/reviews.csv
Waiting on bqjob_r2164450280b400b_00000199ca89b29a_1 ... (1s) Current status: DONE   

Row counts:
/bi

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


In [3]:
%%bigquery --project directed-bongo-471119-d1
SELECT 'users' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.reviews`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,users,20600
1,movies,2080
2,watch_history,210000
3,reviews,30900
4,search_logs,53000
5,recommendation_logs,104000


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

autodetect is acceptable for exploratory data analysis, simple schemas, or rapid prototyping. Explicit schemas should be used for production pipelines and with complex data types to ensure data quality and consistency. It also creates clear doucumentation of your data's structure.

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [17]:
%%bigquery --project directed-bongo-471119-d1
-- Users: % missing per column
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `directed-bongo-471119-d1.netflix.users`
)
SELECT n,
       ROUND(100*miss_region/n,2) AS pct_missing_region,
       ROUND(100*miss_plan/n,2)   AS pct_missing_plan_tier,
       ROUND(100*miss_age/n,2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,20600,0.0,0.0,11.93


In [7]:
%%bigquery --project directed-bongo-471119-d1
-- Calculate the percentage of missing subscription_plan by state_province
SELECT
    state_province,
    COUNT(*) AS n,
    ROUND(100.0 * COUNTIF(subscription_plan IS NULL) / COUNT(*), 2) AS pct_missing_subscription_plan
FROM
    `directed-bongo-471119-d1.netflix.users`
GROUP BY
    state_province
ORDER BY
    pct_missing_subscription_plan DESC
-- Missing at Random (MAR) occurs if the missingness of subscription_plan
-- is related to the state_province, but not to the value of subscription_plan itself.
-- Analyzing missingness by state_province helps identify potential MAR patterns.

Query is running:   0%|          |

Downloading:   0%|          |

,state_province,n,pct_missing_subscription_plan
0,Alberta,620,0.0
1,Arizona,722,0.0
2,British Columbia,618,0.0
3,California,726,0.0
4,Florida,706,0.0
5,Georgia,718,0.0
6,Illinois,710,0.0
7,Indiana,748,0.0
8,Manitoba,598,0.0
9,Maryland,702,0.0


In [ ]:
# # EXAMPLE (from LLM) — Missingness profile (commented)
# # -- Users: % missing per column
# # WITH base AS (
# #   SELECT COUNT(*) n,
# #          COUNTIF(region IS NULL) miss_region,
# #          COUNTIF(plan_tier IS NULL) miss_plan,
# #          COUNTIF(age_band IS NULL) miss_age
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.users`
# # )
# # SELECT n,
# #        ROUND(100*miss_region/n,2) AS pct_missing_region,
# #        ROUND(100*miss_plan/n,2)   AS pct_missing_plan_tier,
# #        ROUND(100*miss_age/n,2)    AS pct_missing_age_band
# # FROM base;

In [ ]:
# # EXAMPLE (from LLM) — MAR by region (commented)
# # SELECT region,
# #        COUNT(*) AS n,
# #        ROUND(100*COUNTIF(plan_tier IS NULL)/COUNT(*),2) AS pct_missing_plan_tier
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.users`
# # GROUP BY region
# # ORDER BY pct_missing_plan_tier DESC;

### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [8]:
%%bigquery --project directed-bongo-471119-d1
-- Combine the missingness percentages from previous queries
SELECT 'pct_missing_region' AS metric, pct_missing_region AS percentage FROM (
  SELECT ROUND(100*COUNTIF(state_province IS NULL)/COUNT(*),2) AS pct_missing_region
  FROM `directed-bongo-471119-d1.netflix.users`
)
UNION ALL
SELECT 'pct_missing_plan_tier' AS metric, pct_missing_plan_tier AS percentage FROM (
  SELECT ROUND(100*COUNTIF(subscription_plan IS NULL)/COUNT(*),2) AS pct_missing_plan_tier
  FROM `directed-bongo-471119-d1.netflix.users`
)
UNION ALL
SELECT 'pct_missing_age_band' AS metric, pct_missing_age_band AS percentage FROM (
  SELECT ROUND(100*COUNTIF(age IS NULL)/COUNT(*),2) AS pct_missing_age_band
  FROM `directed-bongo-471119-d1.netflix.users`
);

Query is running:   0%|          |

Downloading:   0%|          |

,metric,percentage
0,pct_missing_region,0.00
1,pct_missing_plan_tier,0.00
2,pct_missing_age_band,11.93


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

Age has the most missing values of these columns from the user table. These values could be MCAR if age is not required when creating account and many users don't want to bother to put theirs in or do not feel that it is safe to disclose their birthdate. However, this could also be MNAR if users of a certain age do not want to disclose that information.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [9]:
%%bigquery --project directed-bongo-471119-d1
-- Report duplicate groups on (user_id, movie_id, action, device_type) with counts (top 20)
SELECT user_id, movie_id, action, device_type, COUNT(*) AS dup_count
FROM `directed-bongo-471119-d1.netflix.watch_history`
GROUP BY user_id, movie_id, action, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,action,device_type,dup_count
0,user_00391,movie_0893,stopped,Laptop,8
1,user_03310,movie_0640,stopped,Smart TV,8
2,user_04506,movie_0244,stopped,Desktop,6
3,user_02028,movie_0037,paused,Desktop,6
4,user_06417,movie_0590,completed,Laptop,6
5,user_01581,movie_0933,stopped,Desktop,6
6,user_03348,movie_0688,paused,Desktop,6
7,user_06799,movie_0458,completed,Desktop,6
8,user_09512,movie_0825,paused,Desktop,6
9,user_09973,movie_0342,stopped,Desktop,6


In [12]:
%%bigquery --project directed-bongo-471119-d1
-- Create a new table watch_history_dedup by keeping one row per duplicate group
-- Duplicates are identified by the combination of user_id, movie_id, action, and device_type.
-- The row to keep is selected based on the highest progress_percentage, then the highest watch_duration_minutes.
CREATE OR REPLACE TABLE `directed-bongo-471119-d1.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, action, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `directed-bongo-471119-d1.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


In [ ]:
# # EXAMPLE (from LLM) — Detect duplicate groups (commented)
# # SELECT user_id, movie_id, event_ts, device_type, COUNT(*) AS dup_count
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history`
# # GROUP BY user_id, movie_id, event_ts, device_type
# # HAVING dup_count > 1
# # ORDER BY dup_count DESC
# # LIMIT 20;

In [ ]:
# # EXAMPLE (from LLM) — Keep-one policy (commented)
# # CREATE OR REPLACE TABLE `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup` AS
# # SELECT * EXCEPT(rk) FROM (
# #   SELECT h.*,
# #          ROW_NUMBER() OVER (
# #            PARTITION BY user_id, movie_id, event_ts, device_type
# #            ORDER BY progress_ratio DESC, minutes_watched DESC
# #          ) AS rk
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history` h
# # )
# # WHERE rk = 1;

### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [13]:
%%bigquery --project directed-bongo-471119-d1
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count FROM `directed-bongo-471119-d1.netflix.watch_history_dedup`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,watch_history,210000
1,watch_history_dedup,99958


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

Duplicates can arise from human error - multiple users imputing the same information to the same table or one user mistakenly entering the same information twice. Errors in data transformation can prompt the system to duplicate data. Parallel processing is the system equivalent of multiple humans entering the same data into a table. Retrying processes that failed midway could also insert duplicate records. They corrupt labels and KPIs because they destroy the accuracy of the information from which they were constructed, causing them to be inaccurate and potentially misleading.

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [14]:
%%bigquery --project directed-bongo-471119-d1
-- Compute IQR bounds for watch_duration_minutes and report % outliers
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `directed-bongo-471119-d1.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-1) AS hi -- Corrected: Use q3-q1 for IQR calculation
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `directed-bongo-471119-d1.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,1779,99958,1.78


In [15]:
%%bigquery --project directed-bongo-471119-d1
-- Create watch_history_robust with minutes_watched_capped at P01/P99
CREATE OR REPLACE TABLE `directed-bongo-471119-d1.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `directed-bongo-471119-d1.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS watch_duration_minutes_capped
FROM `directed-bongo-471119-d1.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after capping
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q
  FROM `directed-bongo-471119-d1.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(watch_duration_minutes_capped, 5) AS q
  FROM `directed-bongo-471119-d1.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,q
0,after,"[4.4, 24.7, 41.6, 61.7, 92.2, 203.6]"
1,before,"[0.2, 24.8, 41.7, 61.5, 91.4, 799.3]"


In [ ]:
# # EXAMPLE (from LLM) — IQR outlier rate (commented)
# # WITH dist AS (
# #   SELECT
# #     APPROX_QUANTILES(minutes_watched, 4)[OFFSET(1)] AS q1,
# #     APPROX_QUANTILES(minutes_watched, 4)[OFFSET(3)] AS q3
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup`
# # ),
# # bounds AS (
# #   SELECT q1, q3, (q3-q1) AS iqr,
# #          q1 - 1.5*(q3-q1) AS lo,
# #          q3 + 1.5*(q3-q1) AS hi
# #   FROM dist
# # )
# # SELECT
# #   COUNTIF(h.minutes_watched < b.lo OR h.minutes_watched > b.hi) AS outliers,
# #   COUNT(*) AS total,
# #   ROUND(100*COUNTIF(h.minutes_watched < b.lo OR h.minutes_watched > b.hi)/COUNT(*),2) AS pct_outliers
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup` h
# # CROSS JOIN bounds b;

In [ ]:
# # EXAMPLE (from LLM) — Winsorize + quantiles (commented)
# # CREATE OR REPLACE TABLE `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_robust` AS
# # WITH q AS (
# #   SELECT
# #     APPROX_QUANTILES(minutes_watched, 100)[OFFSET(1)]  AS p01,
# #     APPROX_QUANTILES(minutes_watched, 100)[OFFSET(98)] AS p99
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup`
# # )
# # SELECT
# #   h.*,
# #   GREATEST(q.p01, LEAST(q.p99, h.minutes_watched)) AS minutes_watched_capped
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup` h, q;
# #
# # -- Quantiles before vs after
# # WITH before AS (
# #   SELECT 'before' AS which, APPROX_QUANTILES(minutes_watched, 5) AS q
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup`
# # ),
# # after AS (
# #   SELECT 'after' AS which, APPROX_QUANTILES(minutes_watched_capped, 5) AS q
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_robust`
# # )
# # SELECT * FROM before UNION ALL SELECT * FROM after;

### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [16]:
%%bigquery --project directed-bongo-471119-d1
SELECT 'watch_history' AS table_name,
       MIN(watch_duration_minutes) AS min_duration,
       APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_duration,
       MAX(watch_duration_minutes) AS max_duration
FROM `directed-bongo-471119-d1.netflix.watch_history_dedup`
UNION ALL
SELECT 'watch_history_robust' AS table_name,
       MIN(watch_duration_minutes_capped) AS min_duration,
       APPROX_QUANTILES(watch_duration_minutes_capped, 2)[OFFSET(1)] AS median_duration,
       MAX(watch_duration_minutes_capped) AS max_duration
FROM `directed-bongo-471119-d1.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,min_duration,median_duration,max_duration
0,watch_history,0.2,51.3,799.3
1,watch_history_robust,4.4,50.9,203.6


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

Capping could be harmful if the outlier represents an accurate data point. It can be very valuable to understand how and why some data points contain extreme values. Decision trees are less sensitive to outliers because of the way they split data. Outliers can only affect the tree if they are included in a decision node, and their effect is limited to that node.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [17]:
%%bigquery --project directed-bongo-471119-d1
-- Summarize flag_binge for watch sessions > 8 hours (480 minutes)
SELECT
  COUNTIF(watch_duration_minutes > 480) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes > 480)/COUNT(*),2) AS pct
FROM `directed-bongo-471119-d1.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct
0,639,99958,0.64


In [19]:
%%bigquery --project directed-bongo-471119-d1
-- Summarize flag_age_extreme if age of user is less than 10 or greater than 100
SELECT
  COUNTIF(age < 10 OR age > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct
FROM `directed-bongo-471119-d1.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_rows,total,pct
0,358,20600,1.74


In [22]:
%%bigquery --project directed-bongo-471119-d1
-- Summarize flag_duration_anomaly for movies where duration_min < 15 or > 480
SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 480) AS titles_over_8h,
  COUNT(*) AS total,
  ROUND(100 * (COUNTIF(duration_minutes < 15) + COUNTIF(duration_minutes > 480)) / COUNT(*), 2) AS pct_duration_anomaly
FROM `directed-bongo-471119-d1.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,titles_under_15m,titles_over_8h,total,pct_duration_anomaly
0,24,22,2080,2.21


In [ ]:
# # EXAMPLE (from LLM) — flag_binge (commented)
# # SELECT
# #   COUNTIF(minutes_watched > 8*60) AS sessions_over_8h,
# #   COUNT(*) AS total,
# #   ROUND(100*COUNTIF(minutes_watched > 8*60)/COUNT(*),2) AS pct
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_robust`;

In [ ]:
# # EXAMPLE (from LLM) — flag_age_extreme (commented)
# # SELECT
# #   COUNTIF(CAST(REGEXP_EXTRACT(age_band, r'\d+') AS INT64) < 10 OR
# #           CAST(REGEXP_EXTRACT(age_band, r'\d+') AS INT64) > 100) AS extreme_age_rows,
# #   COUNT(*) AS total,
# #   ROUND(100*COUNTIF(CAST(REGEXP_EXTRACT(age_band, r'\d+') AS INT64) < 10 OR
# #                     CAST(REGEXP_EXTRACT(age_band, r'\d+') AS INT64) > 100)/COUNT(*),2) AS pct
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.users`;

In [ ]:
# # EXAMPLE (from LLM) — flag_duration_anomaly (commented)
# # SELECT
# #   COUNTIF(duration_min < 15) AS titles_under_15m,
# #   COUNTIF(duration_min > 8*60) AS titles_over_8h,
# #   COUNT(*) AS total
# # FROM `${GOOGLE_CLOUD_PROJECT}.netflix.movies`;

### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [23]:
%%bigquery --project directed-bongo-471119-d1
-- Summarize all anomaly flags in a single query
SELECT 'flag_binge' AS flag_name, ROUND(100*COUNTIF(watch_duration_minutes > 480)/COUNT(*),2) AS pct_of_rows
FROM `directed-bongo-471119-d1.netflix.watch_history_robust`
UNION ALL
SELECT 'flag_age_extreme' AS flag_name, ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct_of_rows
FROM `directed-bongo-471119-d1.netflix.users`
UNION ALL
SELECT 'flag_duration_anomaly' AS flag_name, ROUND(100 * (COUNTIF(duration_minutes < 15) + COUNTIF(duration_minutes > 480)) / COUNT(*), 2) AS pct_of_rows
FROM `directed-bongo-471119-d1.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.64
1,flag_age_extreme,1.74
2,flag_duration_anomaly,2.21


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

The duration anomaly flag for movies shorter than 15m or longer than 8 hours is the most common. I would keep the binge flag. I think there could be valuable insights gained from understanding which categories of users binge and what kinds of content are being binged. The age of a user or the duration of some content is less valuable to me.

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
